In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
import lxml
import requests
import unicodedata

In [22]:
fm_df = pd.read_csv("uluta_fm23data.csv")
# platinum_df = pd.read_csv("platinum22_fm23data.csv")

In [23]:
fm_df.columns = [col.strip() for col in fm_df.columns]

for col in fm_df.select_dtypes("object").columns:
   fm_df[col] = fm_df[col].apply(lambda x: x.strip())

In [24]:
convert_date_format = lambda x: pd.to_datetime(x, format='%d/%m/%Y').strftime('%Y-%m-%d')
fm_df['DoB'] = fm_df['DoB'].apply(lambda x: x.split(" ")[0])
fm_df['DoB'] = fm_df['DoB'].apply(convert_date_format)

In [25]:
fm_df['Height'] = fm_df['Height'].apply(lambda x: x.split(" ")[0])

In [26]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best Pos', 'Acc', 'Aer', 'Agg', 'Agi', 'Ant', 'Bal', 'Bra', 'Cmd',
       'Com', 'Cmp', 'Cnt', 'Cor', 'Cro', 'Dec', 'Det', 'Dri', 'Ecc', 'Fin',
       'Fir', 'Fla', 'Fre', 'Han', 'Hea', 'Jum', 'Kic', 'Ldr', 'Lon', 'L Th',
       'Mar', 'Nat .1', 'OtB', '1v1', 'Pac', 'Pas', 'Pen', 'Pos', 'Pun', 'Ref',
       'TRO', 'Sta', 'Str', 'Tck', 'Tea', 'Tec', 'Thr', 'Vis', 'Wor'],
      dtype='object')

In [27]:
fm_df.loc[323].to_dict()

{'UID': 14023219,
 'Inf': '',
 'Name': 'Guillermo Fernández',
 'DoB': '1991-10-11',
 'Nat': 'ARG',
 'Division': 'Argentine Premier Division',
 'Club': 'Boca',
 'Based': 'Argentina (Premier Division)',
 'Preferred Foot': 'Right',
 'Right Foot': 'Very Strong',
 'Left Foot': 'Reasonable',
 'Position': 'DM, M/AM (C)',
 'Height': '178',
 'Weight': '75 kg',
 'Age': 30,
 'Transfer Value': '€950K - €5.4M',
 'Wage': '€68,500 p/m',
 'AT Apps': '240',
 'AT Gls': '22',
 'Team': '',
 'Caps': 0,
 'Yth Apps': '-',
 'Style': 'Intelligent',
 'Rc Injury': '-',
 'Best Role': 'Deep Lying Playmaker',
 'Best Duty': 'Support',
 'Best Pos': 'M (C)',
 'Acc': 12,
 'Aer': 2,
 'Agg': 9,
 'Agi': 13,
 'Ant': 14,
 'Bal': 12,
 'Bra': 9,
 'Cmd': 2,
 'Com': 3,
 'Cmp': 12,
 'Cnt': 14,
 'Cor': 13,
 'Cro': 13,
 'Dec': 14,
 'Det': 13,
 'Dri': 13,
 'Ecc': 4,
 'Fin': 12,
 'Fir': 14,
 'Fla': 13,
 'Fre': 12,
 'Han': 3,
 'Hea': 7,
 'Jum': 9,
 'Kic': 1,
 'Ldr': 6,
 'Lon': 12,
 'L Th': 5,
 'Mar': 9,
 'Nat .1': 15,
 'OtB': 11,
 '1

In [28]:
# platinum_df.loc[323].to_dict()

{'Name': 'Antony',
 'Position': 'M/AM RC',
 'Age': 22,
 'ca': 148,
 'pa': 165,
 'Nationality': 'Brazil',
 'Club': 'Manchester UFC',
 'Corners': 11,
 'Crossing': 14,
 'Dribbling': 16,
 'Finishing': 12,
 'First Touch': 16,
 'Free Kick Taking': 12,
 'Heading': 12,
 'Long Shots': 13,
 'Long Throws': 10,
 'Marking': 6,
 'Passing': 14,
 'Penalty Taking': 11,
 'Tackling': 5,
 'Technique': 16,
 'Aggressiion': 12,
 'Anticipation': 13,
 'Bravery': 15,
 'Composure': 14,
 'Concentration': 12,
 'Vision': 13,
 'Decision': 12,
 'Determination': 14,
 'Flair': 16,
 'Leadership': 9,
 'Off The Ball': 14,
 'Position.1': 6,
 'Teamwork': 12,
 'Work Rate': 12,
 'Acceleration': 15,
 'Agility': 16,
 'Balance': 13,
 'Jumping Reach': 7,
 'Natural Fitness': 15,
 'Pace': 14,
 'Stamina': 13,
 'Strength': 11,
 'Stability': 11,
 'Foul': 15,
 'Contest performance': 10,
 'Injury': 7,
 'diversity': 11,
 'Aerial Reach': 3,
 'Command Of Area': 3,
 'Communication': 1,
 'Eccentricity': 1,
 'Handling': 3,
 'Kicking': 2,
 'On

<br><br><br><br>
# Merged_DF

In [29]:
selected_leagues = ['English Premier Division',
 'Ligue 1 Uber Eats',
 'Spanish First Division',
 'Bundesliga',
 'Italian Serie A',
 'Eredivisie',
 'Portuguese Premier League',
 'Turkish Super League']

In [30]:
fm_df[fm_df.Division.isin(selected_leagues)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8460 entries, 42332 to 189312
Data columns (total 74 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   UID             8460 non-null   int64 
 1   Inf             8460 non-null   object
 2   Name            8460 non-null   object
 3   DoB             8460 non-null   object
 4   Nat             8460 non-null   object
 5   Division        8460 non-null   object
 6   Club            8460 non-null   object
 7   Based           8460 non-null   object
 8   Preferred Foot  8460 non-null   object
 9   Right Foot      8460 non-null   object
 10  Left Foot       8460 non-null   object
 11  Position        8460 non-null   object
 12  Height          8460 non-null   object
 13  Weight          8460 non-null   object
 14  Age             8460 non-null   int64 
 15  Transfer Value  8460 non-null   object
 16  Wage            8460 non-null   object
 17  AT Apps         8460 non-null   object
 18  AT Gls 

<br><br><br><br>
# Team Names Converter from fminside.com

In [43]:
url_df = fm_df.drop_duplicates(subset=["Club"], keep="first")
url_df.loc[:, 'UID'] = url_df['UID'].apply(lambda text: str(text))
url_df = url_df[["Club", "UID"]]

In [44]:
def Player_to_URL(player_id):
    slug = player_id + "-x-x"
    return f"https://fminside.net/players/3-fm-23/{slug}"

In [45]:
def Convert_Team_Name_From_URL(player_id, club):
    url = Player_to_URL(player_id)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")
    club_name_element = soup.find('div', class_='meta').find('ul').find_all('li')[0].text
    club_name = club_name_element.strip()
    return {"fm":club,"normal":club_name}

In [46]:
team_mapping_array = []
raw_names = url_df.to_dict()

In [47]:
uids = url_df["UID"]
clubs = url_df["Club"]

In [40]:
# for uid,club in zip(uids,clubs):
#     new_entry = Convert_Team_Name_From_URL(uid, club)
#     print(new_entry)
#     team_mapping_array.append(new_entry)

In [51]:
fminside_normal_names = pd.read_csv("fminside_team_names.csv")
merged_df = pd.merge(fm_df, fminside_normal_names, how='left', left_on='Club', right_on='fm')
merged_df['Club'] = merged_df['normal'].where(merged_df['Club'] == merged_df['fm'], merged_df['Club'])
merged_df.drop(['fm', 'normal'], axis=1, inplace=True)

In [63]:
merged_df.query("Club == 'AC Milan'")

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,TRO,Sta,Str,Tck,Tea,Tec,Thr,Vis,Wor,Unnamed: 0
73770,142173,Inj,Zlatan Ibrahimović,1981-10-03,SWE,Italian Serie A,AC Milan,Italy (Serie A),Either,Very Strong,...,1,8,18,3,14,19,3,15,7,44.0
73775,67211695,,Theo Hernández,1997-10-06,FRA,Italian Serie A,AC Milan,Italy (Serie A),Left,Weak,...,3,16,13,14,12,13,2,12,13,44.0
73781,34000647,,Olivier Giroud,1986-09-30,FRA,Italian Serie A,AC Milan,Italy (Serie A),Left,Reasonable,...,2,12,18,7,19,14,2,13,15,44.0
73784,43298481,,Sandro Tonali,2000-05-08,ITA,Italian Serie A,AC Milan,Italy (Serie A),Right,Very Strong,...,2,14,12,14,16,15,2,14,16,44.0
73803,18045347,,Divock Origi,1995-04-18,BEL,Italian Serie A,AC Milan,Italy (Serie A),Right,Very Strong,...,1,13,15,8,13,14,1,12,15,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,2000209244,Yth,Gaetano Manto,2006-06-21,ITA,Italian Serie A,AC Milan,Italy (Serie A),Right Only,Very Strong,...,3,10,2,5,6,11,3,9,7,44.0
99923,2000209239,Yth,Andrea De Bonis,2006-03-16,ITA,Italian Serie A,AC Milan,Italy (Serie A),Right,Very Strong,...,3,2,3,12,3,1,2,1,4,44.0
100631,2000091624,Loa,Hugo Cuenca,2005-01-08,PAR,Italian Serie A,AC Milan,Italy (Serie A),Left Only,Weak,...,2,8,6,5,9,13,1,12,10,44.0
101141,2000209243,Yth,Alessandro Guffanti,2006-02-20,ITA,Italian Serie A,AC Milan,Italy (Serie A),Right Only,Very Strong,...,1,2,6,12,3,1,2,3,4,44.0
